In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, SimpleRNN,Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
# Load the dataset
df = pd.read_csv('../../dataset/Dataset_Iov.csv')  

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [53]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(165420, 1, 6) (165420,) (41355, 1, 6) (41355,)


In [54]:
print(X_train[:3])

[[[-1.83296655  0.67532932 -0.4296399  -0.49175573 -0.76073707
   -0.35360533]]

 [[ 0.54556369 -1.48075905  2.32753058  3.39563549 -0.76073707
   -0.35360533]]

 [[ 0.54556369 -1.48075905  2.32753058  1.30242483 -0.76073707
   -0.35360533]]]


# Model

In [55]:
# drop_out=float(input("Enter the dropout rate: "))
# print("Dropout rate is: ", drop_out)

In [56]:
def build_deep_rnn_bi_lstm():
    model = Sequential()

    # Input Layer
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    model.add(SimpleRNN(units=100, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(units=100, activation='relu', return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(units=100)))
    model.add(Dropout(0.2))

    # Output Layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the Model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])


    return model


In [57]:
model = build_deep_rnn_bi_lstm()
import time
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=128,
    verbose=1,
)
end_time = time.time()

Epoch 1/50
 289/1293 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - loss: 0.1874 - mae: 0.1411 - mse: 0.0618

KeyboardInterrupt: 

In [ ]:
# Calculate total training time
total_training_time = end_time - start_time  # in seconds
epochs = 50 # e.g., 50
steps_per_epoch = 2500# e.g., 2000

# Total steps
total_steps = epochs * steps_per_epoch
# Calculate time per step
time_per_step = total_training_time / total_steps  # in seconds
time_per_step_ms = time_per_step * 1000  # Convert to milliseconds
# Calculate time per epoch
time_per_epoch = total_training_time / epochs

# Display results
print(f"{time_per_epoch:.2f}s, {time_per_step_ms:.2f} ms/step")
print(f"Total training time: {total_training_time:.2f}s")


# Make predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'(MSE): {mse:.4E}')

# Calculate RMSE
rmse = np.sqrt(mse)

print(f'(RMSE): {rmse:.4E}')


#ACCUARCY
y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100}')


27.61s, 11.04 ms/step
Total training time: 1380.49s
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
(MSE): 7.7022E-15
(RMSE): 8.7762E-08
Accuracy: 100.0


In [ ]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()